In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy

# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

from utils.calcium import calcium


Autosaving every 180 seconds


<ipython-input-1-e4476ecac24a>:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


/home/cat/.conda/envs/donato/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:

# input directory where Suite2p outputs matlab Fall.mat and all .npy files
# input calcium traces .csv file from Inscopix
data_dir = '/media/cat/4TB/donato/maria/binarization_tests/'

#
#fname = '/media/cat/4TB/donato/maria/binarization_tests/DON-015071/20230710/DON015071-RINST-20230710.csv'

#
animal_id = 'DON-015071'

# initialize calcium object and load suite2p data
c = calcium.Calcium(data_dir, animal_id)
c.verbose = True                          # outputs additional information during processing
c.fname_inscopix = None
c.data_type = '1p'    # '2p' or '1p'

# set flags to save matlab and python data
c.save_python = True         # save output as .npz file 
c.save_matlab = False         # save output as .mat file

# manual thresholds for spike detection
c.dff_min = 0.05                  # min %DFF for [ca] burst to considered a spike (default 5%) overwrites percentile threshold parameter
c.percentile_threshold = 0.9999   # this is pretty fixed, we don't change it; we want [ca] bursts that are well outside the "physics-caused"noise
c.maximum_std_of_signal = 0.08     # if std of signal is greater than this, then we have a noisy signal and we mostly exclude the cell
                                  #    - this is a very important flag! come see me if you don't understand it

#
c.inscopix_post_update = False    # Change this parameter to True for updated Inscopix software post ~July 2023

#
c.binarize_data()

print (" ... DONE ...")

 Sessions for animal:  DON-015071
(0)   20230811
(1)   20230710
(a)   All sessions
Please select a session to process:
Processing sesssion: 20230811


BINARIZING:  20230811
self.session_name:  20230811
cells:  (347, 22520)
F times:  (22520,)


model filter: remove bleaching or trends: 100%|██████████| 347/347 [00:02<00:00, 153.91it/s]
348it [00:01, 296.62it/s]              
binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 347/347 [00:00<00:00, 1250.47it/s]
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 347/347 [00:00<00:00, 709.93it/s]


...saving data...
 ... DONE ...


In [53]:
# #############################################
# ######### LOAD SUITE2P AND BINARIZE #########
# #############################################

# # input directory where Suite2p outputs matlab Fall.mat and all .npy files
# # input calcium traces .csv file from Inscopix
# fname = '/media/cat/4TB/donato/maria/binarization_tests/DON015071_RIST4_IDAS6_20230811/DON015071_IDAS6_20230811_RIST.csv'

# data_dir = '/media/cat/4TB/donato/maria/binarization_tests/'
# animal_id = 'DON-015071'


# c = calcium.Calcium(data_dir, 
#                     animal_id,)
# c.verbose = True             # outputs additional information during processing
# c.recompute = True           # recomputes binarization and other processing steps; False: loads from previous saved locations
# #c.data_dir = os.path.split(fname)[0]
# c.parallel_flag = True

# # Load inscopix data
# c.save_python = True         # save output as .npz file
# c.save_matlab = True         # save output as .mat file
# c.fname_inscopix = fname
# c.load_inscopix()
# c.inscopix_flag = True


# c.recompute_binarization = True           # recomputes binarization and other processing steps; False: loads from previous saved locations
# c.data_dir = os.path.split(fname)[0]

# # set flags to save matlab and python data
# c.save_python = True         # save output as .npz file 
# c.save_matlab = True         # save output as .mat file

# ###############################################
# ##### PARAMETERS FOR RUNNING BINARIZATION #####
# ###############################################
# c.min_width_event_onphase = c.sample_rate//2 # set minimum withd of an onphase event; default: 0.5 seconds
# c.min_width_event_upphase = c.sample_rate//4 # set minimum width of upphase event; default: 0.25 seconds

# ############# PARAMTERS TO TWEAK ##############
# #     1. Cutoff for calling somthing a spike:
# #        This is stored in: std_Fluorescence_onphase/uppohase: defaults: 1.5
# #                                        higher -> less events; lower -> more events
# #                                        start at default and increase if data is very noisy and getting too many noise-events
# #c.min_thresh_std_onphase = 2.5      # set the minimum thrshold for onphase detection; defatul 2.5
# #c.min_thresh_std_upphase = 2.5      # set the minimum thershold for uppohase detection; default: 2.5

# #     2. Filter of [Ca] data which smooths the data significantly more and decreases number of binarzied events within a multi-second [Ca] event
# #        This is stored in high_cutoff: default 0.5 to 1.0
# #        The lower we set it the smoother our [Ca] traces and less "choppy" the binarized traces (but we loose some temporal precision)
# c.high_cutoff = 0.5              

# #     3. Removing bleaching and drift artifacts using polynomial fits
# #        This is stored in detrend_model_order
# c.detrend_model_order = 1 # 1-3 polynomial fit
# c.detrend_model_type = 'mode' # 'mode', 'polynomial'

# #
# c.mode_window = None #*30 # 120 seconds @ 30Hz 

# ################################################
# ########### RUN BINARIZATION STEP ##############
# ################################################
# # 
# # double check that we have set the STD thrshold at a reasonable level to catch biggest/highest SNR bursts
# c.show_plots = True

# #
# c.sample_rate = 20
# c.min_width_event_onphase = c.sample_rate   # the onphase needs to be at least 
# c.min_width_event_upphase = c.sample_rate//3 # the upphase needs to be at least 1/3 of a second
# c.recompute_binarization = True

# # Set dynamic threshold for binarization using percentile of fluorescence fit to mode
# c.dff_min = 0.1                     # set the minimum dff value to be considered an event; required for weird negative dff values
#                                     #   that sometimes come out of inscopix data
# c.percentile_threshold = 0.99999
# c.use_upphase = True

# #
# print ("fluorescence shapes: ", c.F.shape)
# #c.F = c.F[:,:1300*30]
# c.show_plots =False
# c.remove_ends = False                     # delete the first and last x seconds in case [ca] imaging had issues
# c.detrend_filter_threshold = 0.001        # this is a very low filter value that is applied to remove bleaching before computing mode
# c.mode_window = 30*30  # None: compute mode on entire time; Value: sliding window based - baseline detection # of frames to use to compute mode

# # for inscopix lower this value; general range is 0.03 to 0.01 
# c.maximum_std_of_signal = 0.03

# #
# c.moment_flag = True
# c.moment = 2
# c.moment_threshold = 0.01   # note this value of 0.01 generally works, but should look at the moment_distribution.png to make sure it's not too high or low
# c.moment_scaling = 0.5        # if "bad" cell above moment_throesld, moment-scaling is the DFF above which we 
#                             #    consider [ca] to be a spike 

# c.binarize_fluorescence()

# # generate standard randomized plots:
# c.save_sample_traces(spacing=10,
#                      scale=10)

# #
# save_image = True
# c.show_rasters(save_image)


In [99]:
###############################################################
######### LOAD BINARZIED DATA AND VISUALIZE CELL ACTIVITY #####
###############################################################

# 
fname = '/home/cat/Downloads/binarized_traces.npz'

c = calcium.Calcium()
c.fname = fname
c.data_dir = os.path.split(fname)[0]
c.load_binarization()

# pick a random cell to visuzlie
cell_id = 43    # 235, 124, 161, 91, 43
# 
scale = 1

#
c.plot_cell_binarization(cell_id, scale)


In [5]:
######################################################
############# SHOW RASTERS ###########################
######################################################

fname = '/media/cat/4TB/donato/nathalie/binarization_tests/bad/binarized_traces.npz'

c = calcium.Calcium()
c.fname = fname
c.data_dir = os.path.split(fname)[0]
c.load_binarization()


c.show_rasters()

In [27]:
#d = np.load('/home/cat/data/nathalie/binarized_traces.npz',allow_pickle=True)


cell_id = 238
plt.figure()
#plt.plot(DFF[cell_id])
plt.plot(c.F[cell_id])
#temp = (c.F-c.f0s[:,None])/c.f0s[:,None]
#plt.plot([0,c.F.shape[1]],[c.f0s[cell_id],c.f0s[cell_id]],'r--')
#plt.plot(temp[cell_id])

# plt.plot(c.dff[cell_id ])
# plt.plot(c.F_filtered[cell_id])
plt.show()

In [3]:

cell_id = 230

cell_ids = np.arange(180,350,1)

n_bursts = []
for cell_id in cell_ids:

    try:
        temp = c.F_upphase_bin[cell_id]
    except:
        continue
    diff = temp[1:]-temp[:-1]
    idx = np.where(diff==1)[0]
    n_bursts.append(idx.shape[0])

plt.figure()
plt.hist(n_bursts, bins=np.arange(0,100,1))

plt.show()


In [80]:
print (c.F_detrended.shape)

# plot the histogram of values for the first 150 cells
split = 180
plt.figure(figsize=(15,10))

# plot histogram for every cell in c.F_detrended
skew = []
kurt = []
mom = []
ax=plt.subplot(2,3,1)
for k in range(c.F_detrended.shape[0]):
    temp = np.histogram(c.F_detrended[k], bins=np.arange(-0.5,0.5,0.01))

    plt.plot(temp[1][1:],
             temp[0],
             alpha=.25,
             c='black' if k<split else 'red',
             label='Cells: 0-180 ' if k==0 else '')
             

    # Compute skewness
    skewness = scipy.stats.skew(temp[0])

    # Compute kurtosis
    kurtosis = scipy.stats.kurtosis(temp[0],fisher=False)

    # compute moment
    moment = scipy.stats.moment(temp[0],moment=2)

    skew.append(skewness)
    kurt.append(kurtosis)
    mom.append(moment)

plt.legend()
plt.xlabel("[ca] values")


#
plt.title("histogram of values for all cells")


#
ax=plt.subplot(2,3,2)

# plot histogram of skew and kurt
temp = np.histogram(skew[:split], bins=np.arange(-10, 10, .25))
plt.plot(temp[1][1:],
            temp[0],
            alpha=.5,
            c='black')
# same for the values > split
temp = np.histogram(skew[split:], bins=np.arange(-10, 10, .25))
plt.plot(temp[1][1:],
            temp[0],
            alpha=.5,
            c='red')
plt.title("skewness")

#
ax=plt.subplot(2,3,3)

# plot histogram of skew and kurt
temp = np.histogram(kurt[:split], bins=np.arange(-10, 100, 1))
plt.plot(temp[1][1:],
            temp[0],
            alpha=.5,
            c='black')
# same for the values > split
temp = np.histogram(kurt[split:], bins=np.arange(-10, 100, 1))
plt.plot(temp[1][1:],
            temp[0],
            alpha=.5,
            c='red')

plt.title("kurtosis")

# same but now for the product of skew and kurt
ax=plt.subplot(2,3,4)

# plot histogram of skew and kurt
temp = np.histogram(mom[:split], bins=np.arange(0, 1E7, 2E5))
plt.plot(temp[1][1:],
            temp[0],
            alpha=.5,
            c='black')
# same for the values > split
temp = np.histogram(mom[split:], bins=np.arange(0, 1E7, 2E5))
plt.plot(temp[1][1:],
            temp[0],
            alpha=.5,
            c='red')
plt.title("moment (2nd)")

# plot the same but for product of kurt, skew and moment
ax=plt.subplot(2,3,5)

# first normalize the values
kurt = np.array(kurt)/np.max(kurt)
skew = np.array(skew)/np.max(skew)
mom = np.array(mom)/np.max(mom)

#
temp = kurt*skew*mom

# plot histogram of skew and kurt
temp2 = np.histogram(temp[:split], bins=np.arange(-0.1, 1, .01))
plt.plot(temp2[1][1:],
            temp2[0],
            alpha=.5,
            c='black')
# same for the values > split
temp2 = np.histogram(temp[split:], bins=np.arange(-0.1, 1, .01))
plt.plot(temp2[1][1:],
            temp2[0],
            alpha=.5,
            c='red')

plt.title("Product of all 3 (normalized)")
#plt.semilogy()
plt.show()




(251, 36027)


In [58]:
#

c.moment_values = np.ones(c.F_detrended.shape[0])
for k in range(c.F_detrended.shape[0]):
    c.moment_values[k] = temp = scipy.stats.moment(c.F_detrended[k], moment=2)


# plot the histogram of the moement values
plt.figure()
temp = np.histogram(c.moment_values,bins=np.arange(-0.1,0.1,0.00125))
plt.plot(temp[1][1:],temp[0]/np.max(temp[0]),label='moment values')
#print (temp[0])

#
# mu, sigma = scipy.stats.norm.fit(
#                                 c.moment_values
#                                 )
# print (mu, sigma)

df, loc, scale = scipy.stats.t.fit(c.moment_values)

# Create an x-axis range for plotting
x = np.linspace(-0.1,0.1,
                1000)

# Compute the corresponding y-values for the fitted distribution
y = scipy.stats.t.pdf(x, df=df, loc=loc, scale=scale)

dist = scipy.stats.t(df=df, loc=loc, scale=scale)

# Calculate the x value of the 95th percentile
x_95th = dist.isf(0.10)
print (x_95th)

# Plot the fitted Gaussian distribution
plt.plot(x, y/np.max(y), 'r-', label="Gaussian Fit")

# plot vertical line at percentile 95
plt.axvline(x=x_95th,
            c='black',
            label='95% percentile')

#plt.xlim(0.,0.01)
plt.legend()
plt.show()

    



0.008148539183374909
